### Initial Checks

In [1]:
!nvidia-smi

Sat May 11 20:10:42 2024       
+---------------------------------------------------------------------------------------+
| NVIDIA-SMI 546.33                 Driver Version: 546.33       CUDA Version: 12.3     |
|-----------------------------------------+----------------------+----------------------+
| GPU  Name                     TCC/WDDM  | Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp   Perf          Pwr:Usage/Cap |         Memory-Usage | GPU-Util  Compute M. |
|                                         |                      |               MIG M. |
|=========================================+======================+======================|
|   0  NVIDIA GeForce RTX 3060 Ti   WDDM  | 00000000:0B:00.0  On |                  N/A |
|  0%   51C    P8              28W / 220W |   1231MiB /  8192MiB |     12%      Default |
|                                         |                      |                  N/A |
+-----------------------------------------+----------------------+--

In [2]:
!dir

 Volume in drive D is Depolama1
 Volume Serial Number is 166C-5C6C

 Directory of d:\OKUL\YL\Dersler\BLG_561E_Deep_Learning\term_project\UAV_Detection

08.05.2024  19:19    <DIR>          .
08.05.2024  19:19    <DIR>          ..
08.05.2024  20:59             3.377 .gitignore
10.05.2024  22:47    <DIR>          datasets
16.04.2024  23:53                32 README.md
11.05.2024  20:10            39.693 train_pipeline.ipynb
28.04.2024  23:30               803 troubleshoot_notes.txt
28.04.2024  23:27               667 usage_examples_notes.txt
08.05.2024  20:35    <DIR>          yolov9
               5 File(s)         44.572 bytes
               4 Dir(s)  164.213.391.360 bytes free


### Configuration

In [3]:
#-- Helper Functions
def now():
    from datetime import datetime
    return datetime.now().strftime("%Y-%m-%d_%H-%M-%S")


In [4]:
%cd "D:\OKUL\YL\Dersler\BLG_561E_Deep_Learning\term_project\UAV_Detection"

#-- Config
DATASET_NAME    = "combinedset12" # "combinedset12" # "dummy_dataset1" # "set1"
BASE_MODEL      = "yolov9-c"

PRETRAINED      = f"{BASE_MODEL}.pt" 
# PRETRAINED      = f"uav_best.pt" 
MODEL_CFG_FILE  = f"{BASE_MODEL}_uav"
DATA_CFG_FILE   = f"uav_{DATASET_NAME}"
HYP_FILE        = f"hyp.scratch-high"

import yaml
config = {
    "train"   : f"../datasets/{DATASET_NAME}/train/images/",
    "val"     : f"../datasets/{DATASET_NAME}/valid/images/",
    "test"    : f"../datasets/{DATASET_NAME}/test/images/",
    "nc"      : 1,
    # "names"   : {0: 'UAV'}
    "names"   : ['UAV']
}
 
with open(f"yolov9/data/{DATA_CFG_FILE}.yaml", "w") as file:
    yaml.dump(config, file, default_flow_style=False)

print("yaml configurations set:")
print(f"{DATASET_NAME   = }")
print(f"{BASE_MODEL     = }")
print(f"{PRETRAINED     = }")
print(f"{MODEL_CFG_FILE = }")
print(f"{DATA_CFG_FILE  = }")
print(f"{HYP_FILE       = }")

d:\OKUL\YL\Dersler\BLG_561E_Deep_Learning\term_project\UAV_Detection
yaml configurations set:
DATASET_NAME   = 'combinedset12'
BASE_MODEL     = 'yolov9-c'
PRETRAINED     = 'yolov9-c.pt'
MODEL_CFG_FILE = 'yolov9-c_uav'
DATA_CFG_FILE  = 'uav_combinedset12'
HYP_FILE       = 'hyp.scratch-high'


### HYPER-PARAMETERS

In [10]:
#-- set hyperparameters 
IMG_SIZE        = 480 # 360 # 640
BATCH_SIZE      = 8 # 16 # 8
EPOCHS          = 30
WORKERS         = 8
FREEZE          = 0 
PATIENCE        = 5 # early stop after PATIENCE epochs
OPTIMIZER       = "Adam" # ['SGD', 'Adam', 'AdamW', 'LION']

train_time = now()
RUN_NAME = f"t{train_time}_{MODEL_CFG_FILE}_size{IMG_SIZE}_ep{EPOCHS}_batch{BATCH_SIZE}_freeze{FREEZE}_dataset-{DATASET_NAME}_optim{OPTIMIZER}"

print(RUN_NAME)

t2024-05-11_21-57-54_yolov9-c_uav_size480_ep30_batch8_freeze0_dataset-combinedset12_optimAdam


### Train

In [11]:
import torch
torch.cuda.empty_cache()
print("empty_cache")
# !nvidia-smi
!nvidia-smi --query-gpu=name,memory.total,memory.used --format=csv

empty_cache
name, memory.total [MiB], memory.used [MiB]
NVIDIA GeForce RTX 3060 Ti, 8192 MiB, 1521 MiB


In [12]:
print("init")

%cd yolov9


# import os
# os.environ["KMP_DUPLICATE_LIB_OK"]="TRUE" # Error #15: Initializing libiomp5md.dll, but found libiomp5md.dll already initialized

!wandb disabled

print("begin train")

!python train_dual.py \
 --name {RUN_NAME} \
 --epochs {EPOCHS} \
 --workers {WORKERS} \
 --device 0 \
 --batch-size {BATCH_SIZE} \
 --img {IMG_SIZE} \
 --freeze {FREEZE} \
 --patience {PATIENCE} \
 --optimizer {OPTIMIZER} \
 --weights weights/{PRETRAINED} \
 --cfg models/detect/{MODEL_CFG_FILE}.yaml \
 --data data/{DATA_CFG_FILE}.yaml \
 --hyp data/hyps/{HYP_FILE}.yaml 


init
[WinError 2] Sistem belirtilen dosyayı bulamıyor: 'yolov9'
d:\OKUL\YL\Dersler\BLG_561E_Deep_Learning\term_project\UAV_Detection\yolov9
W&B disabled.
begin train


train_dual: weights=weights/yolov9-c.pt, cfg=models/detect/yolov9-c_uav.yaml, data=data/uav_combinedset12.yaml, hyp=data/hyps/hyp.scratch-high.yaml, epochs=30, batch_size=8, imgsz=480, rect=False, resume=False, nosave=False, noval=False, noautoanchor=False, noplots=False, evolve=None, bucket=, cache=None, image_weights=False, device=0, multi_scale=False, single_cls=False, optimizer=Adam, sync_bn=False, workers=8, project=runs\train, name=t2024-05-11_21-57-54_yolov9-c_uav_size480_ep30_batch8_freeze0_dataset-combinedset12_optimAdam, exist_ok=False, quad=False, cos_lr=False, flat_cos_lr=False, fixed_lr=False, label_smoothing=0.0, patience=5, freeze=[0], save_period=-1, seed=0, local_rank=-1, min_items=0, close_mosaic=0, entity=None, upload_dataset=False, bbox_interval=-1, artifact_alias=latest
YOLO  2024-4-14 Python-3.10.8 torch-1.13.1 CUDA:0 (NVIDIA GeForce RTX 3060 Ti, 8192MiB)

hyperparameters: lr0=0.01, lrf=0.01, momentum=0.937, weight_decay=0.0005, warmup_epochs=3.0, warmup_momentum=

### Google Drive - Colab

In [ ]:
# #-- zip the output run folder and download it.
# #-- zip -r {out_name} {path_to_folder_to_zip}
# !zip -r /content/{RUN_NAME}.zip /content/yolov9/runs/train/{RUN_NAME}
# from google.colab import files
# files.download(f"/content/{RUN_NAME}.zip")


In [ ]:
# import shutil
# shutil.copy2(f"/content/{RUN_NAME}.zip", 
#              f"/content/drive/MyDrive/BLG_561E_DL/term_project/")

### Memory Check

In [ ]:
!nvidia-smi

In [19]:
# memory footprint support libraries/code
# !ln -sf /opt/bin/nvidia-smi /usr/bin/nvidia-smi
# !pip install gputil
# !pip install psutil
# !pip install humanize

import psutil
import humanize
import os
import GPUtil as GPU

GPUs = GPU.getGPUs()
# XXX: only one GPU on Colab and isn’t guaranteed
gpu = GPUs[0]
def printmem():
    process = psutil.Process(os.getpid())
    print("Gen RAM Free: " + humanize.naturalsize(psutil.virtual_memory().available), " |     Proc size: " + humanize.naturalsize(process.memory_info().rss))
    print("GPU RAM Free: {0:.0f}MB | Used: {1:.0f}MB | Util {2:3.0f}% | Total     {3:.0f}MB".format(gpu.memoryFree, gpu.memoryUsed, gpu.memoryUtil*100, gpu.memoryTotal))
printmem()


Gen RAM Free: 16.7 GB  |     Proc size: 92.0 MB
GPU RAM Free: 6822MB | Used: 1215MB | Util  15% | Total     8192MB
